# Preprocessing 

In [151]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import joblib
import matplotlib.pyplot as plt

In [124]:
# Lets assume the final dataset is the merged school and students dataset and this is the target Student_Performance_Score column, just to test the pipelines.
students_df = pd.read_csv('student.csv')
schools_df = pd.read_csv('school.csv')



In [107]:
schools_df

,Unnamed: 0,Student_ID,Teacher_Student_Ratio,Average_Teacher_Experience_Years,Average_Class_Size,School_Funding_Per_Student,School_Extracurricular_Activities,Parental_Involvement_Score,School_Facilities_Rating,Internet_Access_In_School,School_Distance_From_Home_km,Student_Attendance_Rate,Disciplinary_Actions_Taken,Student_Performance_Score
0,0,93810,22.841652,6,32,29802,NaN,6.987876,7.310185,True,3.539458,94.309591,2,84.625910
1,1,24592,22.841652,13,32,22499,Sports,8.590361,6.107974,True,2.236010,96.323506,0,55.781241
2,2,13278,22.841652,9,32,23274,NaN,9.516444,6.607501,False,2.059882,89.489596,2,34.434954
3,3,46048,22.841652,19,32,15406,Both,5.170510,7.422160,True,0.502030,90.207585,3,55.727495
4,4,42098,22.841652,13,32,24152,Sports,7.325010,7.062171,False,0.525874,83.124602,0,83.546412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,99921,22.841652,12,32,25077,Arts,6.269570,8.331150,False,3.991072,94.251694,3,35.600080
4996,4996,30152,22.841652,14,32,12522,NaN,9.161446,6.198330,False,1.005611,93.933766,0,90.357950
4997,4997,13113,22.841652,6,32,29780,NaN,5.637470,6.976876,False,4.076370,91.507950,2,85.432135
4998,4998,70191,22.841652,5,32,28234,Arts,6.330069,8.612709,True,1.761693,95.815414,3,40.753051


In [108]:
schools_df.drop('Unnamed: 0', axis=1, inplace=True)

In [109]:
schools_df.columns

Index(['Student_ID', 'Teacher_Student_Ratio',
       'Average_Teacher_Experience_Years', 'Average_Class_Size',
       'School_Funding_Per_Student', 'School_Extracurricular_Activities',
       'Parental_Involvement_Score', 'School_Facilities_Rating',
       'Internet_Access_In_School', 'School_Distance_From_Home_km',
       'Student_Attendance_Rate', 'Disciplinary_Actions_Taken',
       'Student_Performance_Score'],
      dtype='object')

In [110]:
schools_df.shape, students_df.shape

((5000, 13), (5000, 18))

In [111]:
matching_ids = students_df[students_df['Student_ID'].isin(schools_df['Student_ID'])]
num_matching_ids = matching_ids['Student_ID'].nunique() 
print(f"Number of matching Student_IDs: {num_matching_ids}")


Number of matching Student_IDs: 272


In [67]:
non_matching_ids_schools = schools_df[~schools_df['Student_ID'].isin(students_df['Student_ID'])]
num_non_matching_schools = non_matching_ids_schools['Student_ID'].nunique()
print(f"Number of Student_IDs in schools_df not in students_df: {num_non_matching_schools}")


Number of Student_IDs in schools_df not in students_df: 4728


In [59]:

merged_df = pd.merge(students_df, schools_df, how='inner', on='Student_ID')

In [60]:
merged_df.columns


Index(['Unnamed: 0', 'Student_ID', 'First_Name', 'Last_Name', 'Gender',
       'Date_of_Birth', 'Admission_Date', 'Class_Section', 'Class_Level',
       'Religion', 'Tribe', 'State_of_Origin', 'Parent_Name',
       'Parent_Occupation', 'Parent_Income_Range', 'Academic_Performance',
       'Attendance_Rate', 'Special_Needs', 'Teacher_Student_Ratio',
       'Average_Teacher_Experience_Years', 'Average_Class_Size',
       'School_Funding_Per_Student', 'School_Extracurricular_Activities',
       'Parental_Involvement_Score', 'School_Facilities_Rating',
       'Internet_Access_In_School', 'School_Distance_From_Home_km',
       'Student_Attendance_Rate', 'Disciplinary_Actions_Taken',
       'Student_Performance_Score'],
      dtype='object')

In [61]:
merged_df

,Unnamed: 0,Student_ID,First_Name,Last_Name,Gender,Date_of_Birth,Admission_Date,Class_Section,Class_Level,Religion,...,Average_Class_Size,School_Funding_Per_Student,School_Extracurricular_Activities,Parental_Involvement_Score,School_Facilities_Rating,Internet_Access_In_School,School_Distance_From_Home_km,Student_Attendance_Rate,Disciplinary_Actions_Taken,Student_Performance_Score
0,9,62187,Kunle,Femi,Male,1983-08-05,2017-03-12,C,SS2,Islam,...,32,22689,Sports,7.681169,6.157973,False,0.648704,92.393336,1,28.698076
1,14,98555,Funke,Bisi,Male,2002-01-07,2017-08-11,A,SS1,Islam,...,32,15646,NaN,6.436336,7.144552,False,3.086747,93.925468,3,67.955508
2,15,73464,Adewunmi,Afolayan,Male,1988-06-12,2020-06-19,D,JSS1,Christianity,...,32,22282,Sports,9.272627,6.993777,True,2.799946,91.994482,0,90.284140
3,28,23577,Morenike,Akinyemi,Male,2003-05-04,2019-05-25,D,SS3,Islam,...,32,10993,Arts,6.992130,8.950560,True,3.630927,91.133497,3,25.963655
4,42,26150,Ajoke,Adeyemi,Male,1987-09-04,2016-03-17,D,JSS1,Islam,...,32,22762,Sports,5.651733,8.765706,False,0.766265,97.301755,0,83.274265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,4948,86556,Salim,Abubakar,Male,2005-04-21,2019-12-17,B,JSS2,Islam,...,32,14343,Both,7.328914,8.643151,False,2.126450,84.104166,0,98.238486
268,4951,35443,Ibrahim,Lamido,Male,2000-03-21,2016-03-10,A,SS2,Islam,...,32,12936,Both,7.921115,8.537813,False,2.978472,92.244997,2,63.371459
269,4962,54020,Saidun,Liman,Female,1996-02-07,2016-04-03,C,SS3,Islam,...,32,21369,Sports,5.052385,6.878103,False,1.301248,93.891548,0,70.356719
270,4987,51973,Ibrahim,Dikko,Male,1985-11-18,2016-06-11,A,SS1,Islam,...,32,17072,NaN,6.373634,6.245178,False,2.405347,97.540168,0,93.034785


In [112]:
def separate_features_target(df, target_column):
    y = df[target_column]
    X = df.drop(columns=[target_column])
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    return numerical_features, categorical_features, X, y


In [123]:
students_df.columns

Index(['Unnamed: 0', 'Student_ID', 'First_Name', 'Last_Name', 'Gender',
       'Date_of_Birth', 'Admission_Date', 'Class_Section', 'Class_Level',
       'Religion', 'Tribe', 'State_of_Origin', 'Parent_Name',
       'Parent_Occupation', 'Parent_Income_Range', 'Academic_Performance',
       'Attendance_Rate', 'Special_Needs'],
      dtype='object')

In [114]:
students_df.columns

Index(['Unnamed: 0', 'Student_ID', 'First_Name', 'Last_Name', 'Gender',
       'Date_of_Birth', 'Admission_Date', 'Class_Section', 'Class_Level',
       'Religion', 'Tribe', 'State_of_Origin', 'Parent_Name',
       'Parent_Occupation', 'Parent_Income_Range', 'Academic_Performance',
       'Attendance_Rate', 'Special_Needs'],
      dtype='object')

In [125]:
students_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [134]:
numerical_features = ['Attendance_Rate']
categorical_features = [
 'Gender',
 'Class_Section',
 'Class_Level',
 'Religion',
 'Tribe',
 'State_of_Origin',
 'Parent_Occupation',
 'Parent_Income_Range',
 'Special_Needs']

In [131]:
def prepare_data(df):
    df = df.drop(columns=['First_Name', 'Last_Name', 'Parent_Name', 'Student_ID'])
    current_year = pd.Timestamp.now().year
    df['Age'] = current_year - pd.to_datetime(df['Date_of_Birth']).dt.year
    df['Years_Since_Admission'] = current_year - pd.to_datetime(df['Admission_Date']).dt.year
    df = df.drop(columns=['Date_of_Birth', 'Admission_Date'])
    
    return df

In [141]:
def preprocessing_pipeline(numerical_features, categorical_features):
    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')), 
        ('scaler', RobustScaler())  
    ])
    
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  
    ])
    
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])
    
    return preprocessor

def preprocess_data(df, numerical_features, categorical_features, target_column):
    df = prepare_data(df)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

    preprocessor = preprocessing_pipeline(numerical_features, categorical_features)
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)
    
    return X_train_preprocessed, X_test_preprocessed, y_train, y_test, preprocessor

target_column = 'Academic_Performance'
X_train, X_test, y_train, y_test, preprocessor = preprocess_data(students_df, numerical_features, categorical_features, target_column)
print(X_train.shape, X_test.shape)


(4000, 684) (1000, 684)


In [136]:
#Function to reduce dimensionality, if necessary
def apply_pca(X_train, X_test, n_components=100):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

X_train_pca, X_test_pca = apply_pca(X_train, X_test, n_components=100)

print(X_train_pca.shape, X_test_pca.shape)


(4000, 100) (1000, 100)


In [ ]:
# Function to get the names of the features after preprocessing 
def get_feature_names(preprocessor, numerical_features, categorical_features):
    numerical_names = numerical_features
    categorical_transformer = preprocessor.named_transformers_['cat']
    categorical_names = categorical_transformer['onehot'].get_feature_names_out(categorical_features)
    all_feature_names = list(numerical_names) + list(categorical_names)
    
    return all_feature_names

X_train_dense = X_train.toarray()
all_feature_names = get_feature_names(preprocessor, numerical_features, categorical_features)
X_train_df = pd.DataFrame(X_train_dense, columns=all_feature_names)

print(X_train_df.head())


In [ ]:
def initialize_models():
    models = {
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
        'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42)
    }

    # Stacking Regressor (combines all models with Ridge as meta-learner)
    models['Stacking'] = StackingRegressor(
        estimators=[
            ('rf', models['RandomForest']),
            ('gb', models['GradientBoosting']),
            ('et', models['ExtraTrees']),
            ('ada', models['AdaBoost'])
        ],
        final_estimator=Ridge()
    )
    
    return models

# Function to train the models
def train_models(models, X_train, y_train):
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)

# Function to make predictions with the models
def make_predictions(models, X_test):
    predictions = {}
    for name, model in models.items():
        predictions[name] = model.predict(X_test)
    return predictions

# Function to evaluate the models using Mean Squared Error
def evaluate_models(models, y_test, predictions):
    mse_scores = {}
    for name, preds in predictions.items():
        mse = mean_squared_error(y_test, preds)
        mse_scores[name] = mse
        print(f"{name} MSE: {mse}")
    return mse_scores

# Main function to run the entire workflow
def run_ensemble_models(X_train, X_test, y_train, y_test):
    models = initialize_models()
    train_models(models, X_train, y_train)
    predictions = make_predictions(models, X_test)
    mse_scores = evaluate_models(models, y_test, predictions)
    
    return mse_scores



# Run the ensemble models and get MSE scores
mse_scores = run_ensemble_models(X_train, X_test, y_train, y_test)


# Tuning the best model according to the evaluation

In [ ]:
#Assuming the best model is a random forest model

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)

# Get the best hyperparameters
best_rf = grid_search_rf.best_estimator_
print("Best Hyperparameters for Random Forest:", grid_search_rf.best_params_)


In [ ]:
feature_importances = best_rf.feature_importances_
importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)
print(importances_df.head(10))

In [ ]:
best_model_preds = best_rf.predict(X_test)
mae = mean_absolute_error(y_test, best_model_preds)
r2 = r2_score(y_test, best_model_preds)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")


Saving the model

In [ ]:
# Save the best-performing model
joblib.dump(best_rf, 'best_random_forest_model.pkl')

